In [17]:
import pandas as pd
import qalsadi.lemmatizer
import nltk
from camel_tools.utils.dediac import dediac_ar
from camel_tools.tokenizers.word import simple_word_tokenize
import csv


In [18]:
data=pd.read_excel(r'E:\DataAnalysis Internship\cima.xlsx')


In [19]:
moviesNames=data.get('اسم الفيلم')
date=data.get('تاريخ العرض')
categories=data.get('تصنيف الفيلم')
duration=data.get("مدة الفيلم (دقيقة)")
synopses=data.get('ملخص')
actors=data.get('تمثيل')
author=data.get("تأليف")
producer=data.get("إنتاج")
cameraMan=data.get("تصوير")
montage=data.get("مونتاج")
decor=data.get("ديكور")
malabes=data.get("ملابس")
music=data.get("موسيقى")
director=data.get("إخراج")
distributor=data.get("توزيع")

In [20]:
import string 
import re

from nltk import word_tokenize

punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

# Arabic stop words with nltk
stop_words= pd.read_excel(r'E:\DataAnalysis Internship\stop_words.xlsx')
stop_words=stop_words.get('stop words')
stop_words =list(stop_words)+ list(nltk.corpus.stopwords.words("arabic"))
stop_words.append('بعد')
stop_words.append('خلال')
stop_words.append('الرغم')
stop_words.append('بها')
stop_words.append('به')
stop_words.append('بينما')
stop_words.append('')
stop_words.append('.')
    

arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def preprocess(text):
    
    '''
    text is an arabic string input
    
    the preprocessed text is returned
    '''
    
    #remove punctuations
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    #remove numbers
    text = re.sub("[0123456789]", '', text)

    # remove Tashkeel
    text = re.sub(arabic_diacritics, '', text)
    
    
    #remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text
  


In [21]:
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer

from camel_tools.disambig.mle import MLEDisambiguator

# instantiate the Maximum Likelihood Disambiguator
mle = MLEDisambiguator.pretrained()

def analysis (text):
    temp=[]
    for i in text:
        # The disambiguator expects pre-tokenized text
        i = simple_word_tokenize(i)

        disambig = mle.disambiguate(i)

        diacritized = [d.analyses[0].analysis['diac'] for d in disambig]
        pos_tags = [d.analyses[0].analysis['pos'] for d in disambig]
        lemmas = [d.analyses[0].analysis['lex'] for d in disambig]
        temp.append(' '.join(lemmas))
    return pd.Series(temp)
# Print the combined feature values extracted above
#for triplet in zip(diacritized, pos_tags, lemmas):
#    print(triplet[0])

In [22]:
def preprocessingData(data):
    data = data.apply(preprocess)
    data=analysis(list(data))
    data = data.apply(preprocess)
    data=analysis(list(data))
    data = data.apply(preprocess)
    return data

In [23]:
synopses=preprocessingData(synopses)


In [24]:
%%time

m1=(list(moviesNames))
d1=list(date)
c1=(list(categories))
d2=list(duration)
s1=(list(synopses))
a1=(list(actors))
a2=(list(author))
p1=(list(producer))
c2=(list(cameraMan))
m2=(list(montage))
d3=(list(decor))
m3=(list(malabes))
m4=(list(music))
d4=(list(director))
d5=(list(distributor))
header = ["اسم الفيلم","تاريخ العرض","تصنيف الفيلم","مدة الفيلم (دقيقة)","ملخص","تمثيل","تأليف","إنتاج","تصوير","مونتاج","ديكور","ملابس","موسيقى","إخراج","توزيع"]
data = []
for i in range(len(synopses)) :
    data.append([m1[i],d1[i],c1[i],d2[i],s1[i],a1[i],a2[i],p1[i],c2[i],m2[i],d3[i],m3[i],m4[i],d4[i],d5[i]])
with open('test.csv', 'w', encoding="utf-8" ,newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

Wall time: 51.9 ms
